# Solving the 1D time-dependent Non-Linear Schrodinger equation using the split-step method

Writing the Non-Linear Schrodinger equation in the form:
\begin{equation}
    \frac{\partial \psi}{\partial t} = i\mathcal{L}\psi+i\mathcal{N}\psi
\end{equation}
Where:
\begin{equation}
    \mathcal{L} = -\frac{1}{2}\frac{\partial^2}{\partial x^2}, \ \ \ \ \ \mathcal{N} = -V(x)+\kappa \ |\psi(x,t)|^2
\end{equation}

We have now included a term which couples the wavefunction evolution to its probability density at that point, through the coupling $\kappa$. This equation can be used to describe non-linearity in fiber optics and Bose-Einstein condensation when a potential $V(x)$ is included.

For an outline of the split-step Fourier method see the [previous notebook](split_step_schrodinger.ipynb)


In [ ]:
import pycav.display as display


## Example Animation

Gaussian wavefunction placed off centre in a harmonic potential well. Two animations below, the first included the non-linear term and the second without it i.e. the solution for the linear Schrodinger equation.

In [ ]:
display.display_animation('nonlinwell.mp4')

In [ ]:
display.display_animation('linwell.mp4')

## Below is the code used to produce this animation

The potential now is a function of position, x, and wavefunction, psi. Below is a defined example

In [ ]:
import numpy as np

import pycav.pde as pde
import matplotlib.pyplot as plt
import matplotlib.animation as anim

def oneD_gaussian(x,mean,std,k0):
    return np.exp(-((x-mean)**2)/(2*std**2)+ 1j*x*k0)/np.sqrt(2*np.pi*std**2)

def V(x,psi):
    V_x = np.zeros_like(x)
    V_x = -150.*np.absolute(psi)**2+0.05*x**2
    return V_x

We must define the spatial domain as well as the time step size and number of steps. Then the initial wavefunction is passed to the split step algorithm. The optional argument, non_linear, must be set to True so that the algorithm knows V is to be both a function of $x$ and $\psi$

In [ ]:
N_x = 2**11
dx = 0.05
x = dx * (np.arange(N_x) - 0.5 * N_x)

dt = 0.01
N_t = 3000

p0 = 1.5
dp2 = p0 * p0 * 1./80
d = 1. / np.sqrt(2 * dp2)

psi_0 = oneD_gaussian(x,3.5*d,d,0.)

psi_x,psi_k,k = pde.split_step_schrodinger(psi_0, dx, dt, V, N_t, x_0 = x[0], non_linear = True)

The time evolution can then be animated showing the expected behaviour. The momentum space wavefunction can also be visualised (here shown in the second subplot).

In [ ]:
real_psi = np.real(psi_x)
imag_psi = np.imag(psi_x)
absl_psi = np.absolute(psi_x)
abs_psik = np.absolute(psi_k)

fig = plt.figure(figsize = (10,10))
ax1 = plt.subplot(211)
line1_R = ax1.plot(x,real_psi[:,0],'b')[0]
line1_I = ax1.plot(x,imag_psi[:,0],'r')[0]
line1_A = ax1.plot(x,absl_psi[:,0],'k')[0]
line_V = ax1.plot(x,0.005*V(x,psi_x[:,0]),'k',alpha=0.5)[0]
ax1.set_ylim((real_psi.min(),real_psi.max()))

ax2 = plt.subplot(212)
line2 = ax2.plot(k,abs_psik[:,1],'k')[0]
ax2.set_ylim((abs_psik.min(),abs_psik.max()))

def nextframe(arg):
    line1_R.set_data(x,real_psi[:,10*arg])
    line1_I.set_data(x,imag_psi[:,10*arg])
    line1_A.set_data(x,absl_psi[:,10*arg])
    line_V.set_data(x,0.005*V(x,psi_x[:,10*arg]))
    line2.set_data(k,abs_psik[:,10*arg])
    
animate = anim.FuncAnimation(fig, nextframe, frames = int(N_t/10), interval = 50)
animate = display.create_animation(animate,temp = True)
display.display_animation(animate)